In [ ]:
# Estatisticas

import robot_color_tracking as track
import numpy as np
import json

# Compara o erro entre o resultado esperado e o erro encontrado
methods = [ 'shapes_one_color','shapes','shapes_colors','colors_naive','colors'] 
timeOfTrack = {}
for m in methods:
    if m == 'shapes':
        tracker = track.GeometricTrack(binaryThreshold= 140, segmentMethod='simple')
    elif m == 'colors_naive':
        tracker = track.ColorTrack(binaryThreshold = 80, hueTolerance = 10,kernel = np.ones((10,10)), nbr_colors=4)
    elif m == 'shapes_colors':
        tracker = track.GeometricTrack(areaBounds = (200, 20000),binaryThreshold= 70, segmentMethod='multipleColors', color=['red', 'yellow', 'blue', 'green'])
    elif m == 'colors':
        tracker = track.HoughColorTrack(binaryThreshold = 80, hueTolerance = 10,param1=20, param2=10, minRadius=0,maxRadius=0, nbr_colors=4)
    elif m == 'shapes_one_color':
        tracker = track.GeometricTrack(areaBounds = (200, 20000),binaryThreshold= 70, segmentMethod='oneColor',color='red')
    elif m == 'achromatic':
        tracker = track.AchromaticTrack(d=30,img_width=500, binaryThreshold= 40, hueTolerance=20, satTolerance=130, kernel = np.ones((2,2)),colors=['blue', 'red', 'springGreen', 'yellow'], debug=True)
    
    data = {}
    error = [{},{},{},{}]
    e = 0
    with open('../algorithm_performance_data/real_positions/'+m+'_positions.json', 'r') as f:
        data = json.load(f)
    imgNum = 0
    place = 0
    counterProgress = 0
    for d in sorted(data):
        print('\r'+m+': '+str(format(100*counterProgress/len(data),'.2f'))+'%', end='')
        counterProgress+=1
        shapes = tracker.getRobotIDs()
        tracker.track('../img/'+m+'/'+d)
        trackedPoses = tracker.getPoses()
        realPoses = data[d]
        i=0
        for pos in realPoses:
            if( len(pos) == 0):
                if shapes[i] in trackedPoses:
                    for cont in range(len(trackedPoses[shapes[i]])):
                        error[place][e] = 'false positive'
                        e+=1
            elif shapes[i] in trackedPoses:
                if len(trackedPoses[shapes[i]]) != 0:
                    pos = np.array(pos)
                    posT = np.array(trackedPoses[shapes[i]])
                    if posT.shape[0]==1:
                        if posT.shape[1] == 2:
                            error[place][e] = (posT - pos).tolist()
                        else:
                            error[place][e] = 'false negative'
                    else:
                        dif = posT - pos
                        argminError= (dif**2).sum(axis=1).argmin()
                        error[place][e] = dif[argminError].tolist()
                        for cont in range(posT.shape[0]-1):
                            e+=1
                            error[place][e] = "false positive"
                else:
                    error[place][e] = 'false negative'  
                e+=1
            else:
                #print(d)
                error[place][e] = 'false negative'
                e+=1
            i+=1
        if(imgNum%8 == 7):
            place+=1
        imgNum+=1

    print('\r'+m+': '+format(100*counterProgress/len(data),'.2f')+'%')
    timeOfTrack[m] = tracker.time
    for i in range(4):
        with open('../algorithm_performance_data/error/error_'+m+'_'+str(i)+'.json', 'w') as f:
            json.dump(error[i], f)
    with open('../algorithm_performance_data/time/time_'+m+'.json', 'w') as f:
            json.dump(timeOfTrack[m], f)

In [20]:
# Estatisticas

import robot_color_tracking as track
import numpy as np
import json

# Compara o erro entre o resultado esperado e o erro encontrado
#methods = ['shapes_one_color','shapes','shapes_colors','colors_naive','colors'] 
methods = ['achromatic']
timeOfTrack = {}
error = {}

def calc_dist_error(calculated_positions, actual_positions):
    actual = np.array(actual_positions)
    calculated = np.array(calculated_positions)
    result = []
    
    for i in range(actual.size):
        if actual.size > 0 and calculated.size > 0:
            dist = calculated-actual[0]
            dist = np.sqrt(dist[:,0]**2+dist[:,1]**2)
            result.append(dist.min())
            calculated = np.delete(calculated, dist.argmin(), 0)
            actual = np.delete(actual, 0, 0)
        else:
            break
    
    false_positives = calculated.shape[0]
    false_negatives = actual.shape[0]
    
    return result, false_positives, false_negatives


for m in methods:
    if m == 'shapes':
        tracker = track.GeometricTrack(binaryThreshold= 140, segmentMethod='simple')
    elif m == 'colors_naive':
        tracker = track.ColorTrack(binaryThreshold = 80, hueTolerance = 10,kernel = np.ones((10,10)), nbr_colors=4)
    elif m == 'shapes_colors':
        tracker = track.GeometricTrack(areaBounds = (200, 20000),binaryThreshold= 70, segmentMethod='multipleColors', color=['red', 'yellow', 'blue', 'green'])
    elif m == 'hough_colors':
        tracker = track.HoughColorTrack(binaryThreshold = 80, hueTolerance = 10,param1=20, param2=10, minRadius=0,maxRadius=0, nbr_colors=4)
    elif m == 'shapes_one_color':
        tracker = track.GeometricTrack(areaBounds = (200, 20000),binaryThreshold= 70, segmentMethod='oneColor',color='red')
    elif m == 'achromatic':
        tracker = track.AchromaticTrack(d=30,img_width=500, binaryThreshold= 40, hueTolerance=20, satTolerance=130, kernel = np.ones((2,2)),colors=['blue', 'red', 'springGreen', 'yellow'])
        id_options = ['green', 'blue', 'red', 'yellow']
    
    with open('../algorithm_performance_data/real_positions/'+m+'_positions.json', 'r') as f:
        data = json.load(f)
    
    path = '../img/'+m+'/'
    counterProgress = 0
    for image in data:
        print('\r'+m+': '+str(format(100*counterProgress/len(data),'.2f'))+'%', end='')
        error[image] = { 'false_negative': 0,
                'false_positive': 0,
                'position_error': {},
                'place': data[image]['place'],
                'tags': data[image]['tags'],
        }
        # Track and store actual and calculated poses
        tracker.track(path+image)
        calc_poses = tracker.getPoses()
        actual_poses = data[image]["position"]
        if m == 'achromatic':
            calc_poses['green'] = calc_poses['springGreen']
            del(calc_poses['springGreen'])
            
        
        for id_name in id_options:
            if id_name in actual_poses:
                if id_name in calc_poses:
                    err, false_positive, false_negative = calc_dist_error(calc_poses[id_name], actual_poses[id_name])
                    error[image]['position_error'][id_name] = err
                    error[image]['false_positive'] = false_positive
                    error[image]['false_negative'] = false_negative
                else:
                    error[image]['false_negative']+= len(actual_poses[id_name])  
            elif id_name in calc_poses:
                error[image]['false_positive']+= len(calc_poses[id_name])
        counterProgress+=1
    print('\r'+m+': '+format(100*counterProgress/len(data),'.2f')+'%')

    with open('../algorithm_performance_data/error/error_'+m+'.json', 'w') as f:
        json.dump(error, f)

achromatic: 100.00%


In [24]:
print(error)
with open('../algorithm_performance_data/error/error_'+m+'.json', 'w') as f:
        json.dump(error, f)

{'IMG_20210107_215439.jpg': {'false_negative': 3, 'false_positive': 0, 'position_error': {'red': []}, 'place': 'night right bulb', 'tags': ['v2']}, 'IMG_20210108_161608.jpg': {'false_negative': 1, 'false_positive': 0, 'position_error': {'green': [556.7228979803159], 'blue': [544.5224782320744], 'red': [], 'yellow': []}, 'place': 'daylight', 'tags': ['v2']}, 'IMG_20210107_212517.jpg': {'false_negative': 0, 'false_positive': 44, 'position_error': {'blue': [8.860661868802072]}, 'place': 'night left bulb', 'tags': ['v2']}, 'IMG_20210108_161725.jpg': {'false_negative': 0, 'false_positive': 0, 'position_error': {'green': [1247.8596037627876], 'blue': [1228.5049338525262], 'red': [], 'yellow': [1239.3811299152821]}, 'place': 'daylight', 'tags': ['v2']}, 'IMG_20210107_212456.jpg': {'false_negative': 0, 'false_positive': 4, 'position_error': {'blue': [28.751413377793554]}, 'place': 'night center bulb', 'tags': ['v2']}, 'IMG_20210107_221552.jpg': {'false_negative': 1, 'false_positive': 7, 'posit

In [ ]:
import json
import numpy as np

# Cálculo de medidas estatísticas

methods = [ 'shapes_one_color','shapes','shapes_colors','colors_naive','colors']
statistics = {}
for m in methods:

    distanceMean = []
    distanceDeviation = []
    falseNegative = []
    falsePositive = []
    detected = []
    

    for cont in range(4):    
        with open('../algorithm_performance_data/error/error_'+ m +'_'+str(cont)+'.json', 'r') as f:
            data = json.load(f)

        #for d in range(len(data)):
        #    data[d] = data[str(d)]
        #    del(data[str(d)])
        #print(data)
        errorNegative=0
        errorPositive=0
        errorDist = np.array([0,0])
        euclDist = []
        
        i=0
        for d in data:
            if data[d]=='false negative':
                errorNegative+=1
            elif data[d] == 'false positive':
                errorPositive += 1
            else:
                errorDist = errorDist + np.array(data[d])
                euclDist.append(np.sqrt((np.array(data[d])**2).sum()))
                i+=1
        #print(i)
        if i != 0:
            errorDist = np.sqrt(((errorDist/i)**2).sum())
        else:
            errorDist = -10
        errorNegative /= len(data)
        errorPositive /= len(data)
        detect = i/len(data)
        if len(euclDist)==0:
                distanceDeviation.append(-10)
        else:
            distanceDeviation.append(np.std(np.array(euclDist)))
        distanceMean.append(errorDist)
        falseNegative.append(errorNegative)
        falsePositive.append(errorPositive)
        detected.append(detect)
    with open('../algorithm_performance_data/time/time_'+ m +'.json', 'r') as f:
            timeOfProcessing = np.array(json.load(f))
    
    statistics[m]= {'distanceDeviation': distanceDeviation,'distanceMean':distanceMean,
                    'falseNegative':falseNegative, 'falsePositive':falsePositive, 'detected':detected,
                    'time': timeOfProcessing}


        #print('----------------'+str(cont))
        #print(statistics[0]['distanceMean'])
        #print(errorNegative)
        #print(errorPositive)
        #print(detected)
        #print(len(data))